<h1>Process Data</h1>
<h4>Haley Johnson</h4>

<p>Notebook to extract, combine and transform data</p>

In [1]:
import pandas as pd
import numpy as np
import os 

<h2>Load HTML File</h2>

In [2]:
html = pd.read_html("happiest_cities.html")

In [3]:
happiness_df = html[0]

In [5]:
happiness_df = happiness_df.rename(columns = {'City': 'Location'})

happiness_df['State'] = happiness_df['Location'].apply(lambda s: s.split(",")[-1])
happiness_df['City'] = happiness_df['Location'].apply(lambda s: s.split(",")[0])

happiness_df = happiness_df.drop(columns = ['Location'])

<h2>Load Transportation Data</h2>

In [7]:
locations_df = pd.read_csv("EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv")

In [10]:
list(locations_df.columns)

['OBJECTID',
 'GEOID10',
 'GEOID20',
 'STATEFP',
 'COUNTYFP',
 'TRACTCE',
 'BLKGRPCE',
 'CSA',
 'CSA_Name',
 'CBSA',
 'CBSA_Name',
 'CBSA_POP',
 'CBSA_EMP',
 'CBSA_WRK',
 'Ac_Total',
 'Ac_Water',
 'Ac_Land',
 'Ac_Unpr',
 'TotPop',
 'CountHU',
 'HH',
 'P_WrkAge',
 'AutoOwn0',
 'Pct_AO0',
 'AutoOwn1',
 'Pct_AO1',
 'AutoOwn2p',
 'Pct_AO2p',
 'Workers',
 'R_LowWageWk',
 'R_MedWageWk',
 'R_HiWageWk',
 'R_PCTLOWWAGE',
 'TotEmp',
 'E5_Ret',
 'E5_Off',
 'E5_Ind',
 'E5_Svc',
 'E5_Ent',
 'E8_Ret',
 'E8_off',
 'E8_Ind',
 'E8_Svc',
 'E8_Ent',
 'E8_Ed',
 'E8_Hlth',
 'E8_Pub',
 'E_LowWageWk',
 'E_MedWageWk',
 'E_HiWageWk',
 'E_PctLowWage',
 'D1A',
 'D1B',
 'D1C',
 'D1C5_RET',
 'D1C5_OFF',
 'D1C5_IND',
 'D1C5_SVC',
 'D1C5_ENT',
 'D1C8_RET',
 'D1C8_OFF',
 'D1C8_IND',
 'D1C8_SVC',
 'D1C8_ENT',
 'D1C8_ED',
 'D1C8_HLTH',
 'D1C8_PUB',
 'D1D',
 'D1_FLAG',
 'D2A_JPHH',
 'D2B_E5MIX',
 'D2B_E5MIXA',
 'D2B_E8MIX',
 'D2B_E8MIXA',
 'D2A_EPHHM',
 'D2C_TRPMX1',
 'D2C_TRPMX2',
 'D2C_TRIPEQ',
 'D2R_JOBPOP',
 'D2R_WR

In [12]:
locations_df.rename(columns = {"GEOID10": "2010_fips_code", "GEOID20" : "2018_fips_code", 
                    "STATEFP": 'state_fips_code', "COUNTYFP": "county_fips_code", 
                    "Ac_Total": 'total_acres', 'Ac_Land': 'land_acres', 'Ac_Water': 'water_acres', 
                    "Ac_Unpr": "unprotected_acres", "CountHU": "total_housing_units", "HH": "households", 
                    "P_WrkAge": 'percent_working_age', 'AutoOwn0': 'owns_no_cars', 
                    'Pct_AO0': 'percent_owns_no_cars', "AutoOwn1": 'owns_one_car', "Pct_AO1": "percent_owns_one_car",
                    "AutoOwn2p": "owns_two_plus_cars", "Pct_AO2": 'percent_owns_2_plus_cars'})

,OBJECTID,2010_fips_code,2018_fips_code,state_fips_code,county_fips_code,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,...,D5DRI,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area
0,1,4.811300e+11,4.811300e+11,48,113,7825,4,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.184697,0.000476,0.137707,6,14,15,17,14.000000,3110.360820,2.978361e+05
1,2,4.811300e+11,4.811300e+11,48,113,7825,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.323221,0.000801,0.231868,3,10,12,14,10.833333,3519.469110,4.849451e+05
2,3,4.811300e+11,4.811300e+11,48,113,7825,3,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.314628,0.000736,0.213146,1,1,7,17,8.333333,1697.091802,1.067059e+05
3,4,4.811300e+11,4.811300e+11,48,113,7824,1,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.229821,0.000708,0.205018,16,10,17,17,15.666667,2922.609204,4.818284e+05
4,5,4.811300e+11,4.811300e+11,48,113,7824,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.164863,0.000433,0.125296,4,7,11,14,10.166667,3731.971773,6.876848e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220735,220736,7.803100e+11,7.803100e+11,78,30,961000,2,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,20,1,7.333333,3414.446949,3.355857e+05
220736,220737,7.803100e+11,7.803100e+11,78,30,961000,3,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,20,1,7.333333,2421.025608,2.924305e+05
220737,220738,7.803100e+11,7.803100e+11,78,30,961000,5,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,20,1,7.333333,1955.909418,1.619395e+05
220738,220739,7.803100e+11,7.803100e+11,78,30,960700,3,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,10,1,4.000000,16896.768870,1.038966e+07
